In [12]:
from numpy import hstack
from numpy import zeros
from numpy import ones
from numpy.random import rand
from numpy.random import randn
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from matplotlib import pyplot as plt
from math import pi

import numpy as np
from numpy import random

import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions


In [13]:
latent_dim = 5
n_outputs = 1
model = Sequential()
model.add(Dense(15, activation='relu', kernel_initializer='he_uniform', input_dim=latent_dim))
model.add(Dense(n_outputs, activation='linear'))


In [14]:
def pm0(x, m , s ):
      return tf.math.exp(-0.5*((x-m)/s)**2);

m_data =  2
s_data =  1
cte_init = 1/(np.sqrt(2*pi)*s_data)

cte = cte_init

batch_size = 100
#def custom_loss(layer):

latent_dim =  5

x_input = randn(latent_dim * batch_size)
# reshape into a batch of inputs for the network
x_train = x_input.reshape(batch_size, latent_dim)
y_predit = model.predict (x_train)
    # Create a loss function that adds the MSE loss to the mean of all squared activations of a specific layer
def nce_loss(y_true,y_pred):
        
    # first fit kde on y- to get pn 
    #y = model.predict (x_train)
    f = lambda y: tfd.Independent(tfd.Normal(loc=y, scale=np.std(y)))
    #n = y_pred.shape[0]
    n = batch_size
    pn = tfd.MixtureSameFamily(
    mixture_distribution=tfd.Categorical(
    probs=[1 / n] * n),
    components_distribution=f(y_predit))
    
    # 
   
    p_n = pn.prob( y_pred) 
    p_m = pm0(y_pred, m_data, s_data) # noise (generator) density 
    temp = tf.math.reduce_mean( p_m)
    temp1 = tf.math.reduce_mean( p_n)
    proba = p_n/(cte*p_m+p_n)
    indexes =  tf.where(tf.math.is_nan(p_n/(cte*p_m+p_n)))
    
    tf.print ("confusion p_m" , temp)
    tf.print ("confusion p_n" , temp1)
    tf.print (" indexes " , indexes)
    tf.print(" shape" ,indexes.shape)
    tf.print("type" , type(indexes.shape[0]))
    tf.print("size", tf.size(indexes))
    try:
        #if (tf.size(indexes)>0):
        #if indexes.shape[0] >0:
        proba_nan= tf.gather(p_n, indexes[0])

        tf.print ("pn where its nan " , proba_nan)
    except:
        pass

    tf.print("minimum de pm ",tf.math.reduce_min(p_m))
    tf.print("minimum de pn ",tf.math.reduce_min(p_n))
    
    
    result = tf.math.reduce_mean( tf.math.log(p_n/(cte*p_m+p_n)))
    #result = tf.math.reduce_mean(p_m)
    #print("log likelihood 1", result)
    tf.print("\n log likelihood 2", result)
    return result


    # Create a loss function that adds the MSE loss to the mean of all squared activations of a specific layer
def nce_metric(y_predit):
        
    # first fit kde on y- to get pn 
    #y = model.predict (x_train)
    f = lambda y: tfd.Independent(tfd.Normal(loc=y, scale=np.std(y)))
    #n = y_pred.shape[0]
    n = batch_size
    pn = tfd.MixtureSameFamily(
    mixture_distribution=tfd.Categorical(
    probs=[1 / n] * n),
    components_distribution=f(y_predit))
    
    # 
    
    #print(np.mean(y_predit))
    p_n = pn.prob( y_predit) 
    p_m = pm0(y_predit, m_data, s_data) # noise (generator) density 
    #print("pn ", np.mean(p_n))
    #print("p_m ", np.mean(p_m))
    temp = tf.math.reduce_mean( (p_n/(cte*p_m+p_n)))
    print ("confusion proba " , temp)
    return temp

In [15]:


model.compile(loss= nce_loss, optimizer='adam')


In [18]:
n_epochs = 500
for i in range(n_epochs):

    x_input = randn(latent_dim * batch_size)
    # reshape into a batch of inputs for the network
    x_train = x_input.reshape(batch_size, latent_dim)
    y_train = np.ones(batch_size)
    y_predit = model.predict (x_train)
    
    print( " ############ ", i , " ###############")
    print(  " y BEFORE train ", np.mean(y_predit))
    #nce_metric(y_predit)
    
    with tf.GradientTape() as tape:
        loss = nce_loss(y_train, model(x_train))

    #print(tape.gradient(loss, model.trainable_weights))
    
    print("x shape", x_train.shape)
    print("y shape", y_train.shape)
    tf.print(" model evaluate " ,model.evaluate(x_train, y_train))
    #print(" y a til des nan before" ,np.sum(np.isnan(model.get_weights()[0])))
    tf.print(" model train on batch " , model.train_on_batch(x_train, y_train, ))
    
    y_predit = model.predict (x_train)
    
    print( "y after train ", np.mean(y_predit))
    
    #print(" y a til des nan after" ,np.sum(np.isnan(model.get_weights()[0])))
    
    #nce_metric(y_predit)
    
    #print(" variance " , np.std(y_predit))
   
    
    if np.sum(np.isnan(model.get_weights()[0])) != 0:
        break

 ############  0  ###############
 y BEFORE train  0.6202763
confusion p_m 0.379387975
confusion p_n 0.235525861
 indexes  []
 shape TensorShape([0, 2])
type <class 'int'>
size 0
minimum de pm  0.0163746402
minimum de pn  0.0295870602

 log likelihood 2 -0.479186803
x shape (100, 5)
y shape (100,)
confusion p_m 0.343787611
confusion p_n 0.251674265
 indexes  []
 shape TensorShape([None, 2])
type <class 'NoneType'>
size 0
 32/100 [========>.....................] - ETA: 0s

InvalidArgumentError:  slice index 0 of dimension 0 out of bounds.
	 [[node loss/dense_7_loss/strided_slice (defined at <ipython-input-14-fbc17585decd>:50) ]] [Op:__inference_distributed_function_1810]

Errors may have originated from an input operation.
Input Source operations connected to node loss/dense_7_loss/strided_slice:
 loss/dense_7_loss/Where (defined at <ipython-input-14-fbc17585decd>:39)

Function call stack:
distributed_function


In [227]:
p_m =0.213281333
p_n= 0.058389347

p_n/(cte*p_m+p_n)


0.4069616496901851

In [200]:
x_train = x_input.reshape(batch_size, latent_dim)
y_predit = model.predict (x_train)
# Create a loss function that adds the MSE loss to the mean of all squared activations of a specific layer
nce_metric(y_predit)

confusion proba  tf.Tensor(0.9876856, shape=(), dtype=float32)


<tf.Tensor: shape=(), dtype=float32, numpy=0.9876856>

In [80]:
pm0(0.016741264, 2, 1)

<tf.Tensor: shape=(), dtype=float32, numpy=0.13992375>

In [91]:
y_predit = model.predict (x_train)
print(np.mean(y_predit))

-1.8780168


In [92]:
pm0(-1.8780168, m_data,s_data)

<tf.Tensor: shape=(), dtype=float32, numpy=0.00054239994>

In [121]:
with tf.GradientTape() as tape:
    loss = nce_loss(y_train, model(x_train))

tape.gradient(loss, model.trainable_weights)

confusion  tf.Tensor(nan, shape=(), dtype=float32)


[<tf.Tensor: shape=(5, 15), dtype=float32, numpy=
 array([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan]], dtype=float32)>,
 <tf.Tensor: shape=(15,), dtype=float32, numpy=
 array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan], dtype=float32)>,
 <tf.Tensor: shape=(15, 1), dtype=float32, numpy=
 array([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], dtype=float32)>,
 <tf.Tensor:

In [190]:
tensor = tf.range(10)
tf.print(tensor)

[0 1 2 ... 7 8 9]


In [214]:
0.0311287586/0.041129888

0.7568403444230142

In [215]:
cte

0.3989422804014327